In [5]:


import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm
import numpy as np

# loading the model

In [7]:
device = torch.device('cuda:1')

In [3]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512,512)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [32]:
model_state_dict = torch.load('rn18_pass_100epochs.pth',map_location='cuda:1')

In [33]:
model.load_state_dict(model_state_dict['rn18'])

<All keys matched successfully>

In [34]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
])

In [35]:
ds_train = datasets.CIFAR100(root='./data', train=True, download=True, transform=preprocess)
ds_test  = datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)

Files already downloaded and verified
Files already downloaded and verified


In [36]:
class_order = model_state_dict['class_order']

In [37]:
def instances_from_classes(dataset, class_order):
    subset = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label in class_order:
            subset.append(i)
    return subset


In [39]:

set_train = instances_from_classes(ds_train,class_order[:50])
set_test  = instances_from_classes(ds_test, class_order[:50])

In [40]:
#define sklearn test data
def get_embeddings(lmodel,dl):
    lmodel.to(device)
    loop = tqdm(dl)
    lpred = []
    ly = []
    with torch.no_grad():
        for x,y in loop:
            x = x.to(device)
            y = y.to(device)
            pred = lmodel(x)
            lpred = lpred+pred.tolist()
            ly    = ly + y.tolist()
    return lpred,ly

In [41]:

set_train = instances_from_classes(ds_train,class_order[:50])
set_test  = instances_from_classes(ds_test, class_order[:50])

In [42]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_train))
dl_test = torch.utils.data.DataLoader(ds_test,  batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_test))

In [43]:
train_emb,train_y = get_embeddings(model,dl_train)
test_emb,test_y   = get_embeddings(model,dl_test)

  0%|          | 0/391 [00:00<?, ?it/s]

100%|██████████| 79/79 [00:07<00:00, 11.15it/s]


In [44]:
train_emb = np.matrix(train_emb)
train_y   = np.matrix(train_y)
test_emb   = np.matrix(test_emb)
test_y     = np.matrix(test_y)

In [45]:
train_emb.shape, train_y.shape

((25000, 512), (1, 25000))

In [46]:
test_emb.shape,test_y.shape

((5000, 512), (1, 5000))

# training knn

In [47]:
import sklearn
import sklearn.neighbors as neighbors

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
train_emb,train_y = get_embeddings(model,dl_train)
test_emb,test_y   = get_embeddings(model,dl_test)
train_emb = np.matrix(train_emb)
train_y   = np.matrix(train_y)
test_emb   = np.matrix(test_emb)
test_y     = np.matrix(test_y)

In [ ]:
def eval_emb(train_emb,train_y,test_emb):
    clf = neighbors.KNeighborsClassifier(n_neighbors=1,weights='distance')
    clf.fit(train_emb,train_y.T)
    pred = clf.predict(test_emb)
    return pred


In [53]:
clf = neighbors.KNeighborsClassifier(n_neighbors=1,weights='distance')

In [54]:
clf.fit(train_emb,train_y.T)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=1, weights='distance')

In [55]:
pred = clf.predict(test_emb)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [56]:
from sklearn.metrics import classification_report

In [57]:
print(classification_report(test_y.T,pred))

              precision    recall  f1-score   support

           0       0.86      0.83      0.85       100
           5       0.36      0.64      0.46       100
           7       0.62      0.60      0.61       100
           8       0.81      0.78      0.80       100
           9       0.81      0.86      0.83       100
          10       0.52      0.53      0.53       100
          14       0.67      0.60      0.63       100
          16       0.62      0.72      0.67       100
          20       0.87      0.81      0.84       100
          21       0.85      0.69      0.76       100
          22       0.68      0.55      0.61       100
          25       0.54      0.53      0.54       100
          30       0.57      0.79      0.66       100
          31       0.62      0.71      0.66       100
          32       0.48      0.61      0.54       100
          33       0.42      0.65      0.51       100
          34       0.67      0.60      0.63       100
          35       0.77    